In [15]:
import numpy as np
import pandas as pd

In [3]:
malaria_data = "/Users/andreaparra/Dropbox/4_Work/DataLamaCovid/projects/mining_malaria/malaria_mining/data/cleaned/malaria_cases/V.1/anonymized_individual_data.csv"
df = pd.read_csv(malaria_data, parse_dates=["date"])
df.head()

,area_type,resolution,confirmed,depto_name,desmovilizados,desplazados,age,type_of_confirmation,ethnic_group,event,...,individual_id,gender,victima_de_violencia_armada,num_cases,depto_id,muni_id,notification_muni_id,residence_muni_id,ocupation_id,resolution_id
0,CABECERA,VIVO (A),CONFIRMADO,Antioquia,NO,SI,31,CONFIRMADO POR LABORATORIO,INDÍGENA,MALARIA FALCIPARUM,...,ff08fc24a07b54abb004f9704d2cce7ce1f20a1a5ead19...,MASCULINO,NO,1,5.0,5120.0,5120,5120.0,OBREROS DE CARGA,VIVO (A)
1,AREA RURAL DISPERSA,VIVO (A),CONFIRMADO,Antioquia,NO,NO,34,CONFIRMADO POR LABORATORIO,"NEGRO, MULATO, AFROCOLOMBIANO O AFRODESCENCIENTE",MALARIA VIVAX,...,d9145536d2d69103aae7b943d05c12963394e4fc204519...,MASCULINO,NO,1,5.0,5101.0,66400,66400.0,NO DEFINIDO,VIVO (A)
2,CENTRO POBLADO,VIVO (A),CONFIRMADO,Antioquia,NO,NO,34,CONFIRMADO POR LABORATORIO,OTRAS ETNIAS,MALARIA FALCIPARUM,...,409173bbbaf62576a699529281cdffb178fbdc76a55f1b...,MASCULINO,NO,1,5.0,5042.0,13244,13244.0,LIMPIADORES DE FACHADAS Y DESHOLLINADORES,VIVO (A)
3,AREA RURAL DISPERSA,VIVO (A),CONFIRMADO,Antioquia,NO,NO,44,CONFIRMADO POR LABORATORIO,OTRAS ETNIAS,MALARIA FALCIPARUM,...,95e8f981deaf558c9cb3dc67089d497d2e8750525ffec8...,FEMENINO,NO,1,5.0,5142.0,5045,5142.0,NO DEFINIDO,VIVO (A)
4,CENTRO POBLADO,VIVO (A),CONFIRMADO,Antioquia,NO,NO,24,CONFIRMADO POR LABORATORIO,OTRAS ETNIAS,MALARIA VIVAX,...,c4eca5bb8520eae012cf9249468ece17bdb83f7f8dc11f...,MASCULINO,NO,1,5.0,5101.0,68101,5101.0,NO DEFINIDO,VIVO (A)


In [66]:
def one_hot_encoding(df, columns_to_exclude=[]):
    # Determine columns of categorical variables
    encoding_dict = {}

    variables_names = df.dtypes.index.to_list()
    categorical_variables = [not (np.issubdtype(i, np.number) or np.issubdtype(i, np.datetime64)) \
        for i in df.dtypes]

    variables_to_encode = []
    for idx, var in enumerate(categorical_variables):
        if var and variables_names[idx] not in columns_to_exclude: variables_to_encode.append(variables_names[idx])


    for var in variables_to_encode:
        translation_dic = {}

        # Assign names to new columns
        new_cols, old_cols = zip(*[(f'{var}_{idx + 1}', i) for idx, i in enumerate(df[var].unique())])
        encoding_dict[var] = dict([(i, j) for i, j in zip(new_cols, old_cols)])
        translation_dic[var] = dict([(j, i) for i, j in zip(new_cols, old_cols)])

        # Create dummy variables
        df_tmp = pd.get_dummies(df[var])
        df_tmp.rename(columns=translation_dic[var], inplace=True)

        # Merge and drop original column 
        df = df.merge(df_tmp, left_index=True, right_index=True)
        df.drop(columns=var, inplace=True)

    return df, encoding_dict

In [67]:
df_encoded, encoding_dict = one_hot_encoding(df, columns_to_exclude=['depto_name', 
                                        'muni_name', 
                                        'notification_muni_name', 
                                        'residence_muni_name', 
                                        'individual_id', 
                                        'depto_id', 
                                        'muni_id', 
                                        'notification_muni_id', 
                                        'residence_muni_id'])

df_encoded.head()

,depto_name,age,date,muni_name,notification_muni_name,residence_muni_name,individual_id,num_cases,depto_id,muni_id,...,ocupation_id_313,ocupation_id_242,ocupation_id_240,ocupation_id_296,ocupation_id_270,ocupation_id_161,ocupation_id_250,resolution_id_2,resolution_id_3,resolution_id_1
0,Antioquia,31,2006-10-01,Cáceres,Cáceres,Cáceres,ff08fc24a07b54abb004f9704d2cce7ce1f20a1a5ead19...,1,5.0,5120.0,...,0,0,0,0,0,0,0,0,0,1
1,Antioquia,34,2006-12-28,Ciudad Bolívar,La Virginia,La Virginia,d9145536d2d69103aae7b943d05c12963394e4fc204519...,1,5.0,5101.0,...,0,0,0,0,0,0,0,0,0,1
2,Antioquia,34,2006-12-25,Santafé De Antioquia,El Carmen De Bolívar,El Carmen De Bolívar,409173bbbaf62576a699529281cdffb178fbdc76a55f1b...,1,5.0,5042.0,...,0,0,0,0,0,0,0,0,0,1
3,Antioquia,44,2006-01-20,Caracolí,Apartadó,Caracolí,95e8f981deaf558c9cb3dc67089d497d2e8750525ffec8...,1,5.0,5142.0,...,0,0,0,0,0,0,0,0,0,1
4,Antioquia,24,2006-11-28,Ciudad Bolívar,Bolívar,Ciudad Bolívar,c4eca5bb8520eae012cf9249468ece17bdb83f7f8dc11f...,1,5.0,5101.0,...,0,0,0,0,0,0,0,0,0,1


In [68]:
encoding_dict

{'area_type': {'area_type_1': 'CABECERA',
  'area_type_2': 'AREA RURAL DISPERSA',
  'area_type_3': 'CENTRO POBLADO'},
 'resolution': {'resolution_1': 'VIVO (A)',
  'resolution_2': 'MUERTO (A)',
  'resolution_3': 'NO DEFINIDO'},
 'confirmed': {'confirmed_1': 'CONFIRMADO'},
 'desmovilizados': {'desmovilizados_1': 'NO', 'desmovilizados_2': 'SI'},
 'desplazados': {'desplazados_1': 'SI', 'desplazados_2': 'NO'},
 'type_of_confirmation': {'type_of_confirmation_1': 'CONFIRMADO POR LABORATORIO',
  'type_of_confirmation_2': 'CONFIRMADO POR NEXO EPIDEMIOLOGICO',
  'type_of_confirmation_3': 'CONFIRMADO POR CLINICA'},
 'ethnic_group': {'ethnic_group_1': 'INDÍGENA',
  'ethnic_group_2': 'NEGRO, MULATO, AFROCOLOMBIANO O AFRODESCENCIENTE',
  'ethnic_group_3': 'OTRAS ETNIAS',
  'ethnic_group_4': 'ROM (GITANO)',
  'ethnic_group_5': 'RAIZAL (SAN ANDRES Y PROVIDENCIA)',
  'ethnic_group_6': 'PALENQUERO DE SAN BASILIO'},
 'event': {'event_1': 'MALARIA FALCIPARUM',
  'event_2': 'MALARIA VIVAX',
  'event_3': '